In [ ]:
# Setting my folder directory
# Spacings are fine!

from os import chdir
path = "C://Users//nelso//Documents//Data Science//Python//Scripts//pySpark"
chdir(path)

In [ ]:
import findspark
findspark.init("C:\devtools\spark-2.4.4-bin-hadoop2.7")
print("Hello! findspark ran fine.")

In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

sc = SparkContext("local", "first app")

# from pyspark.sql import SparkSession

print("SparkContext initialized.")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Testing PySpark dataframes") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

print('SparkSession Initialized')
df = spark.read.load("sampledf.csv",
                     format="csv", sep=",", inferSchema="true", header="true")
df.select("Recipe").show(5)

In [ ]:
##### Reading JSON

json_df = spark.read.json("allrecipes_small.json")
json_df.printSchema()

json_df.createOrReplaceTempView("mytable")

json_df = spark.sql("SELECT ingredients, title FROM mytable")
json_df.show()

In [16]:
#####Using user defined functions
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import re

#reading our stopwords
with open('stopwordlist.txt') as file:
    stopwords = set(file.read().splitlines())
    
print(stopwords)

#User-defined Functions
def removeParenthesis(li):
    output = []
    for text in li:
        text = re.sub(r"\([^)]*\)", "", text)
        text = text.strip()
        output.append(text)
    return output
def removeNumbers(li):
    output = []
    for text in li:
        text = re.sub(r"\d+", "", text)
        text = re.sub(r"[//]", " ", text)
        text = text.strip()
        output.append(text)
    return output
def removeAfterComma(li):
    output = []
    for text in li:
        text = re.sub(r",[^,]*$", "", text)
        text = text.strip()
        output.append(text)
    return output
def removeStop(li, stoplist):
    output = []
    for text in li:
        text_tok = text.split(" ")
        text_output = [tok for tok in text_tok if tok not in stoplist]
        text_output = " ".join(text_output)
        output.append(text_output)
    return output


#create user defined function
udf_removeAfter = udf(removeAfterComma, ArrayType(StringType()))
udf_removeParen = udf(removeParenthesis, ArrayType(StringType()))
udf_removeNumbers = udf(removeNumbers, ArrayType(StringType()))
udf_removeStop = udf(lambda x: removeStop(x,stopwords), ArrayType(StringType()))

print("UDF Successful.")

{'drizzle', 'oz', 'links', 'dashes', 'bunches', 'rolls', 'feet', 'spears', 'crushed', 'london', 'cube', 'prebaked', 'day', 'boxes', 'pkg', 'grinds', 'piece', 'weight', 'bing', 'cups', 't', 'solid', 'market', 'head', 'pods', 'well', 'pieces', 'wraps', 'ball', 'diced', 'splashes', 'muddled', 'archer', 'trays', 'hershey', 'pre', 'slider', 'fresh', 'boneless', 'free', 'cloves', 'hot', 'sleeves', 'square', 'skinned', 'brie', 'box', 'reynolds', 'heads', 'bars', 'tube', 'slabs', 'teaspoons', 'ring', 'twist', 'metal', 'teaspoon', 'or', 'hank', 'container', 'trimmed', 'inch', 'bar', 'sturdy', 'toothpick', 'spot', 'inner', 'mcintosh', 'belgian', 'wedges', 'envelopes', 'ounce', 'bibb', 'bittersweet', 'm', 'bulbs', 'sprigs', 'skewers', 'pinches', 'summer', 'gala', 'dark', 'ears', 'squeezes', 'ounces', 'finely', 'stalk', 'tubs', 'minature', 'packages', 'servings', 'sliced', 'sparkler', 'prepared', 'torn', 'marinated', 'plain', 'oven', 'navel', 'skewer', 'loaves', 'drops', 'to', 'squirts', 'unwrappe

In [23]:
#Feature Engineering to usable df.

cleaned_json_df = json_df.withColumn('rm_paren', udf_removeParen('ingredients')).select('title','ingredients','rm_paren') \
                         .withColumn('rm_after', udf_removeAfter('rm_paren')).select('title','ingredients','rm_paren','rm_after') \
                         .withColumn('rm_numbers', udf_removeNumbers('rm_after')).select('title','ingredients','rm_paren','rm_after','rm_numbers') \
                         .withColumn('rm_complete', udf_removeStop('rm_numbers')).select('title','ingredients','rm_paren','rm_after','rm_numbers','rm_complete')
cleaned_json_df.show(5)

from pyspark.sql.functions import explode, col

mydf = cleaned_json_df.withColumn('exploded',explode('rm_complete')) \
                      .select(col('title').alias('Recipe'),col('exploded').alias('Ingredients'))

mydf.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|         ingredients|            rm_paren|            rm_after|          rm_numbers|         rm_complete|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Basil, Roasted Pe...|[1/2 cup unsalted...|[1/2 cup unsalted...|[1/2 cup unsalted...|[cup unsalted but...|[unsalted butter,...|
|Crispy Cheese Twists|[1/2 cup Parmesan...|[1/2 cup Parmesan...|[1/2 cup Parmesan...|[cup Parmesan che...|[Parmesan cheese,...|
|   Mom's Yeast Rolls|[2 cups hot water...|[2 cups hot water...|[2 cups hot water...|[cups hot water, ...|[water, margarine...|
|Sweet Potato Bread I|[1 1/2 cups white...|[1 1/2 cups white...|[1 1/2 cups white...|[cups white sugar...|[white sugar, veg...|
|         Orange Buns|[1/4 cup butter, ...|[1/4 cup butter, ...|[1/4 cup butter, ...|[cup butter, teas..

In [29]:
# Writing spark df to csv

# mydf.coalesce(1) \
#     .write.option("header", "true").csv("sample_file.csv")

Py4JJavaError: An error occurred while calling o1010.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperatio

In [ ]:
#Stop spark session.

spark.stop()